In [56]:

import datetime
from torch._C import TensorType
from torch.utils.tensorboard import SummaryWriter
from uuid import uuid4 as uu
import shutil
import os
from tensorboard import program
import subprocess
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# plot tensorboard metrics
import os
import matplotlib.pyplot as plt
from tensorflow.python.summary.summary_iterator import summary_iterator


class TensorBoard:
    def __init__(self, tensor_board_root: str):
        self.tensor_board_root = tensor_board_root
        print('init writer')
        self.writer = SummaryWriter(tensor_board_root)

    def init_tensorboard(self):
        self.gen_session_id()
        self.launch_tensorboard()
        self.create_launchTensorboard_script()

    def launch_tensorboard(self):
        print('launch tensorboard')
      
        tensorboard_command = ["tensorboard", "--logdir", self.tensor_board_root]
        cmd = f"start cmd /k {' '.join(tensorboard_command)}"
        print(f"\nExecuting command: {cmd}")
        os.system(cmd)

    def create_launchTensorboard_script(self):
        # Create a script to launch tensorboard, place it in the tensorboard root
        script = f"tensorboard --logdir {self.tensor_board_root}"
        with open(os.path.join(self.tensor_board_root, 'launch_tensorboard.bat'), 'w') as f:
            f.write(script)

    def gen_session_id(self) -> str:
        day = str(datetime.date.today().day).rjust(2, '0')
        month = str(datetime.date.today().month).rjust(2, '0')
        year = str(datetime.date.today().year)[-2:]
        hour = str(datetime.datetime.now().hour).rjust(2, '0')
        minute = str(datetime.datetime.now().minute).rjust(2, '0')
        self.session_id = f'{day}-{month}-{year}_{hour}-{minute}_{str(uu())[-8:]}'
        print(f"Session ID: {self.session_id}")
        return self.session_id


    def write_board(self, epoch: int, train_loss: int, train_accuracy, train_recall, train_precision, test_loss, test_accuracy, test_recall, test_precision, current_learning_rate: float):
      
        # Calc metric and add scalars
        self.writer.add_scalars(f'Loss/{self.session_id}', {'Train': train_loss, 'Test': test_loss}, epoch)

        self.writer.add_scalars(f'Accuracy/{self.session_id}', {'Train': train_accuracy, 'Test': test_accuracy}, epoch)

        self.writer.add_scalars(f'Recall/{self.session_id}', {'Train': train_recall, 'Test': test_recall}, epoch)

        self.writer.add_scalars(f'Precision/{self.session_id}', {'Train': train_precision, 'Test': test_precision}, epoch)

        self.writer.add_scalar(f'Learning Rate/{self.session_id}', current_learning_rate, epoch)

        self.writer.flush()

    def write_confusion_matrix(self, epoch: int, confusion_matrix):
        # log confusion matrix

        plt.figure(figsize=(8, 6))
        sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='Blues')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title('Confusion Matrix')
        self.writer.add_figure('Confusion Matrix', plt.gcf(), global_step=epoch)
        self.writer.flush()

    def write_parameter_histogram(self, model, epoch):
        # log parameter distributions

        for name, param in model.named_parameters():
            plt.hist(param.flatten().detach().cpu().numpy(), bins=50, alpha=0.5, label=name)
            plt.legend()
            plt.title('Parameter Histograms')
            self.writer.add_figure('Parameter Histograms', plt.gcf(), global_step=epoch)
            self.writer.flush()




# ===== Plot TensorBoard Metrics =====


    def extract_tensorboard_data(self, log_dir=None):
        """
        Extracts data from TensorBoard log files.

        Args:
        - log_dir (str): Path to the directory containing TensorBoard event files.

        Returns:
        - data (dict): Dictionary containing metric data {metric_name: [(step1, value1), (step2, value2), ...]}.
        """
        if log_dir is None:
            log_dir = self.tensor_board_root
        data = {}
        events = {}

        for root, _, files in os.walk(log_dir):
            for file in files:
                if file.startswith("events.out"):
                    event_path = os.path.join(root, file)
                    for event in summary_iterator(event_path):
                        events[event.step] = event
                        for value in event.summary.value:
                            if value.tag not in data:
                                data[value.tag] = []
                            data[value.tag].append((event.step, value.simple_value))
        return data, events

    def plot_metrics(self, metric_values, label, title, save_path):
        """
        Plots and saves metrics as images.

        Args:
        - metrics (dict): Dictionary of metrics to plot.
        - save_path (str): Path to save the plot as an image.
        """
        plt.figure(figsize=(12, 8))
        steps, values = zip(*metric_values)
        plt.plot(steps, values, label=label, marker='o', markersize=3)
        plt.legend()
        plt.title(title)
        plt.xlabel('Step')
        plt.ylabel('Value')
        plt.tight_layout()
        plt.grid(True)
        #plt.savefig(f"{save_path}.png")
        plt.show()
        #plt.close()


    def plot_tensorboard_metrics(self, log_dir=None, title='TensorBoard Metrics', xlabel='Step', ylabel='Value', save_path='tensorboard_plot'):
        """
        Extracts and plots TensorBoard metrics as images.

        Args:
        - log_dir (str): Path to the directory containing TensorBoard event files.
        - title (str): Title of the plot.
        - xlabel (str): Label for the x-axis.
        - ylabel (str): Label for the y-axis.
        - save_path (str): Path to save the plot as an image.
        """
        if log_dir is None:
            log_dir = self.tensor_board_root
        metric_data = self.extract_tensorboard_data(log_dir)

        # Split into train and test metrics
        accuracy_values = [(step, value) for sublist in {metric: metric_data[metric] for metric in metric_data.keys() if 'Accuracy' in metric}.values() for step, value in sublist]
        loss_values = [(step, value) for sublist in {metric: metric_data[metric] for metric in metric_data.keys() if 'Loss' in metric}.values() for step, value in sublist]
        recall_values = [(step, value) for sublist in {metric: metric_data[metric] for metric in metric_data.keys() if 'Recall' in metric}.values() for step, value in sublist]
        precision_values = [(step, value) for sublist in {metric: metric_data[metric] for metric in metric_data.keys() if 'Precision' in metric}.values() for step, value in sublist]
        learning_rate_values = [(step, value) for sublist in {metric: metric_data[metric] for metric in metric_data.keys() if 'Learning Rate' in metric}.values() for step, value in sublist]
        
        self.plot_metrics(accuracy_values, 'Accuracy', title, save_path)
        # Call the function for each metric
        # self.plot_metrics(accuracies, 'accuracies_plot')
        # self.plot_metrics(losses, 'losses_plot')
        # self.plot_metrics(recalls, 'recalls_plot')
        # self.plot_metrics(precisions, 'precisions_plot')
        # self.plot_metrics(learning_rates, 'learning_rates_plot')
        
        return metric_data



In [ ]:
accuracy_values = [(step, value) for sublist in {metric: metric_data[metric] for metric in metric_data.keys() if 'Accuracy' in metric}.values() for step, value in sublist]
print(f'Acc Values: {accuracy_values}')

In [4]:
log_dir = "D:\\code\\Bachelorarbeit\\xx_code_BA\\trainingOutput\\2024-05-04_19-56-23_urinal_v2_resnet50\\tensorboard"


In [57]:
import src.trainCNN.tensorboardHandler as tbh
import json


log_dir = "D:\\code\\Bachelorarbeit\\xx_code_BA\\trainingOutput\\2024-05-04_19-56-23_urinal_v2_resnet50\\tensorboard"


tbh = TensorBoard(log_dir)


metric_data, events = tbh.extract_tensorboard_data(log_dir)

with open('metric_data.json', 'w') as f:
    json.dump(metric_data, f)
with open('events.json', 'w') as f:
    json.dump(events, f)

# metric_data = tbh.plot_tensorboard_metrics()

# # dumb metric data as json as file
# import json
# with open('metric_data.json', 'w') as f:
#     json.dump(metric_data, f)


init writer


TypeError: Object of type Event is not JSON serializable

In [108]:
import os
import matplotlib.pyplot as plt
from tensorboard.backend.event_processing import event_accumulator
import pandas as pd
import io
import re


def load_all_events(logdir):
    accumulators = []
    for root, dirs, files in os.walk(logdir):
        for file in files:
            if file.startswith('events.out.tfevents'):
                path = os.path.join(root, file)
                type = root.split('_')[-1]
                ea = event_accumulator.EventAccumulator(path,
                    size_guidance={
                        event_accumulator.SCALARS: 0,
                        event_accumulator.IMAGES: 0,
                        event_accumulator.HISTOGRAMS: 0
                    }
                )
                ea.Reload()
                accumulator = {
                    'ea': ea,
                    'path': path,
                    'type': type
                }
                accumulators.append(accumulator)
    return accumulators


def read_and_plot_tensorboard_data(logdir):
    filter_images = True
    image_tag_whitelist = ['confusion matrix']

    # Prepare directories for plots and images
    plot_dir = os.path.join(logdir, 'plots')
    image_dir = os.path.join(logdir, 'images')
    os.makedirs(plot_dir, exist_ok=True)
    os.makedirs(image_dir, exist_ok=True)

    # Define the regex pattern
    #pattern = r"^(\w+)_(\d{2}-\d{2}-\d{2})_(\d{2}-\d{2})_([a-f0-9]+)_(Train|Test)$"
    pattern = r"^(.+)/(\d{2}-\d{2}-\d{2})_(\d{2}-\d{2})_([a-f0-9]+)$"


    # Initialize the Event Accumulator with only scalars and images
    accumulators = load_all_events(logdir)

    # Extract scalar data
    metrics = {}
    for accumulator in accumulators:
        ea = accumulator['ea']
        data_type = accumulator['type']
        path = accumulator['path']

        scalar_tags = ea.Tags()['scalars']
        for tag in scalar_tags:
            match = re.match(pattern, tag)
            #print(f"\nTag: {tag}")
            if match:
                metric_name, _, _, _ = match.groups()
                #print(f"Metric Name: {metric_name}, Type: {type}")
                if metric_name.lower() in ['accuracy', 'loss', 'recall', 'precision']:
                    events = ea.Scalars(tag)
                    values = [e.value for e in events]
                    steps = [e.step for e in events]
                    full_tag = f'{metric_name}_{data_type}'
                    #full_tag = f'{metric_name}'
                    metrics[full_tag] = pd.DataFrame({'Step': steps, 'Value': values})
                   # print(f"Values: {metrics[full_tag]}")
                elif metric_name.lower() in ['learning rate']:
                    events = ea.Scalars(tag)
                    values = [e.value for e in events]
                    steps = [e.step for e in events]
                    full_tag = f'{metric_name}'
                    metrics[full_tag] = pd.DataFrame({'Step': steps, 'Value': values})
                
    print(f"\nExtracted metrics: {metrics.keys()}")

    # Plot metrics
    for metric in ['Accuracy', 'Loss', 'Recall', 'Precision']:
        train_tag = f'{metric}_Train'
        test_tag = f'{metric}_Test'
        
        if train_tag in metrics and test_tag in metrics:
            plt.figure(figsize=(12, 6))
            plt.plot(metrics[train_tag]['Step'], metrics[train_tag]['Value'], label='Train')
            plt.plot(metrics[test_tag]['Step'], metrics[test_tag]['Value'], label='Test')
            plt.title(f'{metric} over Training Steps')
            plt.xlabel('Training Steps')
            plt.ylabel(metric)
            plt.grid(True)
            # datapoints with label
            for i, txt in enumerate(metrics[train_tag]['Value']):
                plt.annotate(f"{txt:.2f}", (metrics[train_tag]['Step'][i], metrics[train_tag]['Value'][i]), color='gray')
            for i, txt in enumerate(metrics[test_tag]['Value']):
                plt.annotate(f"{txt:.2f}", (metrics[test_tag]['Step'][i], metrics[test_tag]['Value'][i]), color='gray')

            plt.legend()
            plt.savefig(os.path.join(plot_dir, f'{metric}_plot.png'))
            plt.close()
    
    # extract images
    images = {}
    for accumulator in accumulators:
        ea = accumulator['ea']
        path = accumulator['path']
        image_tags = ea.Tags()['images']
        for tag in image_tags:
            if filter_images:
                if tag.lower() in image_tag_whitelist:
                    images[tag] = ea.Images(tag)
            else:
                images[tag] = ea.Images(tag)

        
    print(f"\nExtracted images: {images.keys()}")


    # Export images
    for tag, image in images.items():
        for i, img in enumerate(image):
            img_data = io.BytesIO(img.encoded_image_string)
            img = plt.imread(img_data)
            plt.imshow(img)
            plt.axis('off')
            plt.title(tag)
            plt.savefig(os.path.join(image_dir, f'{tag}_{i}.png'))
            plt.close()

# Example usage:
# read_and_plot_tensorboard_data('./path/to/logdir')

log_dir = "D:\\code\\Bachelorarbeit\\xx_code_BA\\trainingOutput\\2024-05-04_21-20-13_urinal_v2_resnet50\\tensorboard"
read_and_plot_tensorboard_data(log_dir)



Extracted metrics: dict_keys(['Learning Rate', 'Accuracy_Test', 'Accuracy_Train', 'Loss_Test', 'Loss_Train', 'Precision_Test', 'Precision_Train', 'Recall_Test', 'Recall_Train'])

Extracted images: dict_keys(['Confusion Matrix'])


In [ ]:
plt.annotate(f"{txt:.2f}", (metrics[train_tag]['Step'][i], metrics[train_tag]['Value'][i]), color='red')

In [30]:
import re

tag = "Learning Rate/04-05-24_21-20_f133eb55"
pattern = r"^(.+)/(\d{2}-\d{2}-\d{2})_(\d{2}-\d{2})_([a-f0-9]+)$"
match = re.match(pattern, tag)

if match:
    metric_name, date, time, id = match.groups()
    print(f"Metric Name: {metric_name}")
    print(f"Date: {date}")
    print(f"Time: {time}")
    print(f"ID: {id}")
    

Metric Name: Learning Rate
Date: 04-05-24
Time: 21-20
ID: f133eb55


In [24]:
import re

def parse_tag(tag):
    pattern = r"^(\w+)_(\d{2}-\d{2}-\d{2})_(\d{2}-\d{2})_([a-f0-9]+)_(Train|Test)$"
    match = re.match(pattern, tag)
    if match:
        return {
            'Metric': match.group(1),
            'Date': match.group(2),
            'Time': match.group(3),
            'Hash': match.group(4),
            'Type': match.group(5)
        }
    else:
        return None

# Example
tag = "Accuracy_04-05-24_14-43_c6332f5a_Test"
parsed_info = parse_tag(tag)
print(parsed_info)

{'Metric': 'Accuracy', 'Date': '04-05-24', 'Time': '14-43', 'Hash': 'c6332f5a', 'Type': 'Test'}


In [113]:
import datetime
import time

starttime = datetime.datetime.now()
time.sleep(2)
endtime = datetime.datetime.now()
timeelapsed = endtime - starttime
timeelapsed.


TypeError: unsupported format string passed to datetime.timedelta.__format__

In [12]:
import gotifyHandler as gotifyHandler
import time

def get_TimeElapsed(starttime):
    endtime = time.time()
    timeelapsed = endtime - starttime
    string = str(timeelapsed)
    return string

e = 'Test Error'
starttime = time.time()

# wait 1 second
time.sleep(1)

gotifyHandler.send_message(
    title='CNN Training interrupted with error',
    message=f"""Error: {e} \n Time elapsed: {get_TimeElapsed(starttime)}
    """,
)

Sending Gotify message: CNN Training interrupted with error - Error: Test Error 
 Time elapsed: 1.0006401538848877
    


In [27]:
# test try except error handling with traceback
import traceback

try:
    print(1/0)
except Exception as e:
    traceback.print_exc()
    with open('error.txt', 'w') as f:
        traceback.print_exc(file=f)
        f.write('\n\n')
        traceback.print_stack(file=f)




Traceback (most recent call last):
  File "C:\Users\Martin\AppData\Local\Temp\ipykernel_41504\1506348261.py", line 5, in <module>
    print(1/0)
          ~^~
ZeroDivisionError: division by zero


In [7]:
from src.mts_utils import *

end_time = time.time()  # Record end time of epoch
start_time = time.time()  # Record start time of epoch

time.sleep(2)
epoch_time = get_TimeElapsed(start_time)


print(epoch_time)


00:00:02.00


In [11]:
import time

def get_TimeStamp(time):
    return time.strftime('%Y-%m-%d %H:%M:%S')

start_time = time.time()
print(f"Start Time: {start_time}")
time.sleep(2)
print(get_TimeElapsed(start_time))

Start Time: 1717242710.3453424
00:00:02.00


In [12]:
def time_to_string(time):
    return time.strftime('%Y-%m-%d %H:%M:%S')

start_time = time.time()
print(f"Start Time: {time_to_string(start_time)}")

AttributeError: 'float' object has no attribute 'strftime'

In [20]:
import prettytable as pt
import torch


# dummy values as tensors
train_loss = torch.tensor(0.1234)
train_acc = torch.tensor(0.5678)
train_rec = torch.tensor(0.9012)
train_pre = torch.tensor(0.3456)
test_loss = torch.tensor(0.1234)
test_acc = torch.tensor(0.5678)
test_rec = torch.tensor(0.9012)
test_pre = torch.tensor(0.3456)
epoch_loss_test = torch.tensor(0.1234)
epoch_loss_train = torch.tensor(0.5678)

table = pt.PrettyTable()
table.field_names = ["", "Loss", "Accuracy", "Recall", "Precision"]
table.add_row(["Train", f"{epoch_loss_train:.4f}", f"{train_acc:.4f}", f"{train_rec:.4f}", f"{train_pre:.4f}"])
table.add_row(["Test", f"{epoch_loss_test:.4f}", f"{test_acc:.4f}", f"{test_rec:.4f}", f"{test_pre:.4f}"])
print(f"\n{table}")

print(f"Train Accuracy: {train_acc:.4f}")
print(train_acc.item())
print(train_acc)


+-------+--------+----------+--------+-----------+
|       |  Loss  | Accuracy | Recall | Precision |
+-------+--------+----------+--------+-----------+
| Train | 0.5678 |  0.5678  | 0.9012 |   0.3456  |
|  Test | 0.1234 |  0.5678  | 0.9012 |   0.3456  |
+-------+--------+----------+--------+-----------+
Train Accuracy: 0.5678
0.567799985408783
tensor(0.5678)


In [ ]:
test